# RAG con CSVs (Notebook paralelo)
Este cuaderno replica el flujo de `rag_cli.py` pero muestra cada etapa con resúmenes y estadísticas para entender los datos procesados.


In [1]:
pip install -r ../requirements.txt



[notice] A new release of pip is available: 25.2 -> 25.3
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [2]:
import os
import sys
from pathlib import Path
import pandas as pd
from dotenv import load_dotenv
from langchain_openai import ChatOpenAI, OpenAIEmbeddings

load_dotenv()
cwd = Path.cwd()
project_root = cwd
for parent in [cwd] + list(cwd.parents):
    if (parent / 'excel' / 'DrugData.csv').exists():
        project_root = parent
        break
src_dir = project_root / 'src'
if src_dir.exists():
    sys.path.insert(0, str(src_dir))

csv_path = project_root / 'excel' / 'DrugData.csv'
os.environ['RAG_DRUG_CSV'] = str(csv_path)
os.environ.setdefault('RAG_COLLECTION_NAME', 'csv_vademecum')

from rag_modulo3 import (
    DRUG_CSV_PATH,
    COLLECTION_NAME,
)
from rag_modulo3.preparation import (
    load_drug_documents,
    build_vector_store,
)
from rag_modulo3.rag_chain import (
    build_query_rewriter,
    answer_question,
)

print(f'🧾 CSV de medicamentos: {DRUG_CSV_PATH.resolve()}')
print(f'📚 Coleccion en uso: {COLLECTION_NAME}')


🧾 CSV de medicamentos: /Users/fabiansalfatenavarro/Desktop/Diplomando IA/Archivos/M3/Proyecto_Final_V2/excel/DrugData.csv
📚 Coleccion en uso: csv_vademecum


## Inventario de CSVs


In [3]:
print("project_root:", project_root)


project_root: /Users/fabiansalfatenavarro/Desktop/Diplomando IA/Archivos/M3/Proyecto_Final_V2


In [4]:
drug_df = pd.read_csv(DRUG_CSV_PATH)
if drug_df.empty:
    raise RuntimeError('El CSV no tiene registros.')
drug_df.head()


,Drug ID,Drug Name,Generic Name,Drug Class,Indications,Dosage Form,Strength,Route of Administration,Mechanism of Action,Side Effects,Contraindications,Interactions,Warnings and Precautions,Pregnancy Category,Storage Conditions,Manufacturer,Approval Date,Availability,NDC,Price
0,1,Aspirin,Acetylsalicylic Acid,Analgesic,Headache,Tablet,325 mg,Oral,Inhibits prostaglandin synthesis,Stomach irritation,Allergy to NSAIDs,Anticoagulants: increased bleeding risk,Gastric ulcers: risk of bleeding,Category C,Room temperature,PharmaCorp,15-01-2022,OTC,12345678901,5.99
1,2,Amoxicillin,Amoxicillin,Antibiotic,Bacterial Infections,Capsule,500 mg,Oral,Inhibits bacterial cell wall synthesis,Nausea,Allergic reactions,Penicillin: reduced efficacy,Take with food to reduce stomach upset,Category B,Room temperature,MediPharm,20-11-2021,Prescription,23456789012,12.49
2,3,Lisinopril,Lisinopril,Antihypertensive,Hypertension,Tablet,10 mg,Oral,Blocks conversion of angiotensin I to angioten...,Cough,Kidney problems,Diuretics: increased risk of kidney problems,Monitor kidney function regularly,Category D,Room temperature,HealthMeds,05-03-2022,Prescription,34567890123,8.75
3,4,Albuterol,Albuterol,Bronchodilator,Asthma,Inhaler,100 mcg/inhalation,Inhalation,Stimulates beta-2 adrenergic receptors,Tremor,Rapid heart rate,Beta-blockers: decreased effectiveness,Use as needed for acute asthma symptoms,Category C,Room temperature,RespirePharma,10-08-2021,Prescription,45678901234,22.99
4,5,Omeprazole,Omeprazole,Proton Pump Inhibitor,Acid Reflux,Capsule,20 mg,Oral,Reduces stomach acid production,Headache,Abdominal pain,Clopidogrel: reduced effectiveness,Take before a meal,Category C,Room temperature,MediCare,30-05-2022,OTC,56789012345,6.25


## 1. Carga completa de documentos


In [5]:
raw_documents = load_drug_documents()
print(f'Documentos cargados: {len(raw_documents)}')
print('\nEjemplo de metadata base:')
print(raw_documents[0].metadata)
print('\nFragmento del contenido:\n', raw_documents[0].page_content[:500])


Documentos cargados: 220

Ejemplo de metadata base:
{'source': 'DrugData.csv', 'titulo': 'Aspirin', 'categoria': 'Analgesic', 'resumen': 'Headache', 'drug_id': '1', 'generic_name': 'Acetylsalicylic Acid', 'dosage_form': 'Tablet', 'strength': '325 mg', 'route': 'Oral', 'mechanism': 'Inhibits prostaglandin synthesis', 'side_effects': 'Stomach irritation', 'contraindications': 'Allergy to NSAIDs', 'interactions': 'Anticoagulants: increased bleeding risk', 'warnings': 'Gastric ulcers: risk of bleeding', 'pregnancy_category': 'Category C', 'storage': 'Room temperature', 'manufacturer': 'PharmaCorp', 'approval_date': '15-01-2022', 'availability': 'OTC', 'ndc': '12345678901', 'price': '5.99'}

Fragmento del contenido:
 Drug ID: 1
Medicamento: Aspirin
Principio activo: Acetylsalicylic Acid
Grupo terapeutico: Analgesic
Indicaciones: Headache
Presentacion: Tablet
Concentracion: 325 mg
Via de administracion: Oral
Mecanismo de accion: Inhibits prostaglandin synthesis
Efectos secundarios: Stomach i

## 2. Extracción de metadata con GPT-4o


In [6]:
metadata_df = pd.DataFrame([
    {
        'titulo': doc.metadata.get('titulo'),
        'categoria': doc.metadata.get('categoria'),
        'resumen': doc.metadata.get('resumen'),
        'source': doc.metadata.get('source'),
    }
    for doc in raw_documents
])
metadata_df.head()


,titulo,categoria,resumen,source
0,Aspirin,Analgesic,Headache,DrugData.csv
1,Amoxicillin,Antibiotic,Bacterial Infections,DrugData.csv
2,Lisinopril,Antihypertensive,Hypertension,DrugData.csv
3,Albuterol,Bronchodilator,Asthma,DrugData.csv
4,Omeprazole,Proton Pump Inhibitor,Acid Reflux,DrugData.csv


## 3. Semantic chunking y estadísticas


In [7]:
stats_df = pd.DataFrame([
    {
        'titulo': doc.metadata.get('titulo', 'N/A'),
        'source': doc.metadata.get('source'),
        'chars': len(doc.page_content),
        'words': len(doc.page_content.split()),
    }
    for doc in raw_documents
])
print('\nResumen por documento:')
stats_summary = stats_df.groupby(['titulo']).agg(
    palabras_promedio=('words', 'mean'),
)
stats_summary.head()



Resumen por documento:


,palabras_promedio
titulo,
Albuterol,75.666667
Alendronate,74.000000
Allopurinol,74.000000
Alprazolam,71.000000
Amiodarone,71.000000


## 4. Creación / actualización del vector store en Qdrant


In [8]:
vector_store = build_vector_store(raw_documents)
print('Vector store actualizado en Qdrant. Coleccion:', vector_store.collection_name)


Vector store actualizado en Qdrant. Coleccion: csv_vademecum


### Estado de la colección en Qdrant


In [9]:
from rag_modulo3 import COLLECTION_NAME, get_qdrant_client
client = get_qdrant_client()
collection = client.get_collection(COLLECTION_NAME)
params = collection.config.params
vector_size = None
if hasattr(params, "vectors") and isinstance(params.vectors, dict):
    default_vec = params.vectors.get("")
    if default_vec is not None:
        vector_size = getattr(default_vec, "size", None)
elif isinstance(params, dict):
    vectors = params.get("vectors", {})
    if isinstance(vectors, dict):
        default_vec = vectors.get("")
        if isinstance(default_vec, dict):
            vector_size = default_vec.get("size")
print(f"Colección analizada: {COLLECTION_NAME}")
print("Vectores indexados:", collection.points_count)
print("Dimensiones:", vector_size)


Colección analizada: csv_vademecum
Vectores indexados: 220
Dimensiones: 3072


## 5. Consulta de ejemplo con reescritura


In [15]:
# Despues de crear el vector_store:
retriever = vector_store.as_retriever(search_kwargs={'k': 5})
llm = ChatOpenAI(model='gpt-4o', temperature=0.1)
rewrite_chain = build_query_rewriter(llm)
demo_query = 'Para que sirve la Clonazepam y dame su precio?'
response = answer_question(demo_query, vector_store, llm, rewrite_chain)
print(response)


Hola,

El Clonazepam es un medicamento del grupo terapéutico de las benzodiazepinas y está indicado para el tratamiento de la ansiedad. Su mecanismo de acción se basa en la potenciación de la actividad del GABA en el cerebro, lo que ayuda a calmar la actividad neuronal excesiva.

En cuanto a su costo, el precio del Clonazepam varía según la concentración y el fabricante. Por ejemplo, una presentación de 2 mg fabricada por MentalWellness tiene un precio de 5.99, mientras que otra presentación de 0.5 mg fabricada por NeuroRelax cuesta 4.99.

Espero que esta información te sea de ayuda. Si tienes más preguntas, no dudes en consultarme.

Saludos cordiales.

Fuentes consultadas:

- Clonazepam (DrugData.csv)


In [14]:
results = vector_store.similarity_search_with_relevance_scores(demo_query, k=3)
[(getattr(doc, "id", None), score, doc.metadata.get("titulo"), doc.metadata.get("source")) for doc, score in results]


[(None, 0.83880835, 'Clonazepam', 'DrugData.csv'),
 (None, 0.8349852, 'Clonazepam', 'DrugData.csv'),
 (None, 0.8300474, 'Clonazepam', 'DrugData.csv')]

In [11]:
from rag_modulo3.config import load_environment, COLLECTION_NAME
from rag_modulo3.rag_chain import load_vector_store
from langchain_openai import OpenAIEmbeddings
load_environment()
embeddings = OpenAIEmbeddings(model='text-embedding-3-large')
vector_store = load_vector_store(embeddings)
def max_score(query):
    results = vector_store.similarity_search_with_relevance_scores(query, k=3)
    return [(doc.metadata.get('titulo'), score) for doc, score in results]
print(max_score('Ibuprofeno'))
print(max_score('Lisinopril'))
print(max_score('Amoxicilina'))


[('Ibuprofen', 0.8277279), ('Aspirin', 0.765477775), ('Naproxen', 0.76486045)]
[('Lisinopril', 0.81627962), ('Lisinopril', 0.816076665), ('Lisinopril', 0.81434518)]
[('Amoxicillin', 0.84013795), ('Amoxicillin', 0.83818978), ('Amoxicillin', 0.835998)]


In [12]:
# Mostrar hits (vector IDs/score) desde Qdrant
from rag_modulo3.rag_chain import load_vector_store
from langchain_openai import OpenAIEmbeddings

embeddings = OpenAIEmbeddings(model='text-embedding-3-large')
vector_store = load_vector_store(embeddings)

query = 'Amoxicilina'
try:
    results = vector_store.similarity_search_with_relevance_scores(query, k=3)
    hits = [
        {
            'id': getattr(doc, 'id', None),
            'score': score,
            'titulo': doc.metadata.get('titulo'),
            'source': doc.metadata.get('source'),
        }
        for doc, score in results
    ]
except Exception:
    results = vector_store.similarity_search_with_score(query, k=3)
    hits = [
        {
            'id': getattr(doc, 'id', None),
            'score': 1 - min(max(distance or 0.0, 0.0), 1.0),
            'titulo': doc.metadata.get('titulo'),
            'source': doc.metadata.get('source'),
        }
        for doc, distance in results
    ]

hits


[{'id': None,
  'score': 0.84013795,
  'titulo': 'Amoxicillin',
  'source': 'DrugData.csv'},
 {'id': None,
  'score': 0.83818978,
  'titulo': 'Amoxicillin',
  'source': 'DrugData.csv'},
 {'id': None,
  'score': 0.835998,
  'titulo': 'Amoxicillin',
  'source': 'DrugData.csv'}]